**THIS IS ONLY A TEST**

mô hình đang gặp khó khăn trong việc phân loại các mẫu thuộc lớp thiểu số.

# **BIRCH**

Balanced Iterative Reducing and Clustering using Hierarchies

MỤC TIÊU: phân cụm dữ liệu một cách hiệu quả và có khả năng xử lý dữ liệu lớn.

sử dụng một cấu trúc dữ liệu gọi là (BIRCH tree) để tổ chức và phân cụm dữ liệu. BIRCH tree được xây dựng bằng cách kết hợp các điểm dữ liệu thành các nhóm con (subclusters) theo cách đệ quy và tiến hóa dần từ các nhóm con nhỏ đến các nhóm con lớn hơn.

BIRCH tree có hai loại Node chính: node lá (leaf node) và node trong (non-leaf node). node lá chứa các subclusters và được giữ trong bộ nhớ, trong khi node trong chứa thông tin tóm tắt về các subclusters của nó.


## 3 main Steps:


1. **Xây dựng cấu trúc cây:** Dữ liệu được thêm vào **BIRCH tree** một cách tuần tự. Các **subclusters** được hình thành và cập nhật theo từng dữ liệu mới. Nếu một dữ liệu không thể gắn vào bất kỳ **subclusters** nào, một **subclusters** mới sẽ được tạo ra.
2.   **Nén cây:** Sau khi cấu trúc cây được xây dựng, **BIRCH tree** có thể bị phình to do sự tăng trưởng của dữ liệu. Để giảm độ phức tạp của cây, một quá trình nén được thực hiện bằng cách thay thế các **subclusters** bằng các tóm tắt thống kê.
3. **Phân cụm từ cây:** Khi **BIRCH tree** được xây dựng và nén, quá trình phân cụm được thực hiện bằng cách truy xuất các subclusters từ cây và gom nhóm chúng thành các cụm cuối cùng.



## LIBRARY


In [10]:
from sklearn.cluster import Birch
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import pandas as pd

from sklearn.svm import SVC
from sklearn.metrics import classification_report


##Read Data


DATASET from: https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud

The dataset contains transactions made by credit cards in September 2013 by European cardholders.

This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

In [2]:
data = pd.read_csv("/content/drive/MyDrive/DA/creditcard.csv") #path to dataset
data.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.168375e-15,3.416908e-16,-1.379537e-15,2.074095e-15,9.604066e-16,1.487313e-15,-5.556467e-16,1.213481e-16,-2.406331e-15,...,1.654067e-16,-3.568593e-16,2.578648e-16,4.473266e-15,5.340915e-16,1.683437e-15,-3.660091e-16,-1.227390e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [3]:
# Chia dữ liệu thành mẫu huấn luyện và mẫu kiểm tra
train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)

Data Preprocessing

In [5]:
train_data = train_data.drop(['Time'], axis=1)  # Loại bỏ cột 'Time'
test_data = test_data.drop(['Time'], axis=1)

#Chuẩn hóa dữ liệu số học để đảm bảo các biến có cùng phạm vi:
scaler = StandardScaler()
train_data['Amount'] = scaler.fit_transform(train_data['Amount'].values.reshape(-1, 1))
test_data['Amount'] = scaler.transform(test_data['Amount'].values.reshape(-1, 1))

In [6]:
train_data.describe()
test_data.describe()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,28481.000000,28481.000000,28481.000000,28481.000000,28481.000000,28481.000000,28481.000000,28481.000000,28481.000000,28481.000000,...,28481.000000,28481.000000,28481.000000,28481.000000,28481.000000,28481.000000,28481.000000,28481.000000,28481.000000,28481.000000
mean,-0.007086,-0.002514,0.004905,0.003783,-0.001955,-0.003035,-0.004552,-0.000183,-0.006977,0.007666,...,-0.002948,0.006349,-0.002169,-0.002000,0.006552,0.005038,0.002891,-0.001943,-0.002231,0.001615
std,1.978423,1.672097,1.536368,1.418356,1.367306,1.317783,1.216406,1.209943,1.105506,1.106628,...,0.716262,0.723141,0.604830,0.609338,0.521011,0.482026,0.415631,0.336875,0.903214,0.040157
min,-34.148234,-48.060856,-33.680984,-5.560118,-23.669726,-20.869626,-31.197329,-50.420090,-9.462573,-22.187089,...,-22.889347,-8.887017,-22.575000,-2.824849,-3.963454,-2.068561,-22.565679,-11.710896,-0.350184,0.000000
25%,-0.921109,-0.598966,-0.888363,-0.846227,-0.701100,-0.768846,-0.564060,-0.211852,-0.650129,-0.533275,...,-0.228182,-0.539924,-0.164473,-0.355604,-0.311201,-0.322289,-0.071650,-0.053254,-0.327011,0.000000
50%,0.013240,0.065459,0.185381,-0.024544,-0.053946,-0.271935,0.033205,0.020880,-0.054374,-0.087293,...,-0.028402,0.016053,-0.014325,0.042357,0.029078,-0.046561,0.000099,0.010443,-0.263079,0.000000
75%,1.315782,0.801249,1.044630,0.739501,0.611056,0.399630,0.576081,0.324336,0.590223,0.464040,...,0.187241,0.528281,0.145020,0.437848,0.351981,0.247901,0.089816,0.077913,-0.044071,0.000000
max,2.393578,18.902453,9.382558,12.699542,29.016124,16.493227,21.437514,17.052566,15.594995,23.745136,...,27.202839,8.272233,22.083545,3.664552,6.070850,2.991210,9.200883,15.942151,20.403930,1.000000


Apply SMOTE để cân bằng dữ liệu

In [7]:
X_train = train_data.drop(['Class'], axis=1)
y_train = train_data['Class']
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_train, y_train)

# # In ra dữ liệu sau khi áp dụng SMOTE
# resampled_data = pd.concat([pd.DataFrame(X_res), pd.DataFrame(y_res, columns=['Class'])], axis=1)
# resampled_data

Apply BIRCH

In [8]:
birch = Birch(n_clusters=None)  # Đặt n_clusters=None để BIRCH tự động xác định số lượng cụm
birch.fit(X_res)

# Trích xuất nhãn cụm từ BIRCH
cluster_labels = birch.predict(X_res)
n_clusters = len(set(cluster_labels))

print(n_clusters)

169994


Train SVM

In [11]:
svm = SVC()
svm.fit(X_res, y_res)

# Dự đoán trên dữ liệu kiểm tra
X_test = test_data.drop(['Class'], axis=1)
y_test = test_data['Class']
y_pred = svm.predict(X_test)

# Đánh giá kết quả
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99     28435
           1       0.09      0.87      0.16        46

    accuracy                           0.99     28481
   macro avg       0.54      0.93      0.58     28481
weighted avg       1.00      0.99      0.99     28481

